In [ ]:


# Import required packages
from __future__ import print_function, division
import pandas as pd
import numpy as np
from scipy.io import wavfile
import pydub
from pydub import AudioSegment
import librosa
import imageio
import math
import imageio
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import DataLoader
from PIL import Image



# Set the path that has your audio sample and teh .txt file from the detector
data = 'PATH'

# Reading file with start and end time 
input_file = pd.read_csv(data +'/'+ 'ARU18_20120410_090000.txt', encoding='utf-16',delimiter='\t')#Location of the file with Begin and End time Information
#print(input_file)



# Selecting only the colmns of interest i.e. Start Time and End Time
input_file_select = input_file.iloc[:,[2,3]]
#print(input_file_select)




# Splitting audio snippets on the basis of Begin and End Time. The code is creating snippets of 2 sec length with 1 sec overlap

newAudio = AudioSegment.from_wav(data+'/'+'ARU18_20120410_090000.wav') #Location of the main Audio file

# while input_file_select.iloc[len(input_file_select)-1,1] < duration_file:
    
for i in range(0, len(input_file_select)):
  
  t1 = input_file_select.iloc[i,0] * 1000
  t2 = input_file_select.iloc[i,1] * 1000
  leng_audio = t2 - t1
  for j in range(0,math.ceil(leng_audio/2)):
    if (t1+1000) <= t2:
      finalAudio = newAudio[t1:t1+2000]
      t1 = t1+1000
      finalAudio.export(data+'/'+'samples'+'/'+'_'+str(i)+'_'+str(j)+'spec.wav', format='wav') #Exports to a wav file in the current path.






# Generating Spectrograms from the 2 sec segmented snippets of the audio files
  
  # THIS IS THE PATH WHERE THE 2SEC SNIPPETS OF AUDIOFILES GENERATED FROM THE PREVIOUS STEP - LETS CALL ITS PATH_1
path = Path('PATH_1').glob('**/*.wav')#Location of the path of the .wav files
wavs = [str(wavf) for wavf in path if wavf.is_file()]
wavs.sort()

number_of_files=len(wavs)

spk_ID = [wavs[i].split('/')[-1][:-4].lower() for i in range(number_of_files)]

for i in range(number_of_files):
    y, sr = librosa.load(wavs[i], sr=None)
    
    p =librosa.stft(y)
    S = librosa.feature.melspectrogram(y, sr=44100, n_fft=2048, hop_length=512, n_mels=256)
    log_S = librosa.amplitude_to_db(S)
    mean = log_S.mean()
    std = log_S.std()
    S_norm = (log_S - mean) / (std + 1e-6)
    S_min, S_max = S_norm.min(), S_norm.max()
    S_scaled = 255 * (S_norm - S_min) / (S_max - S_min)
    S_scaled = S_scaled.astype(np.uint8)
      

    #Storing the spectrograms of the 2 sec audio snippets at the same location as that of the audio snippets
    save_path = 'PATH_1'
    # # # plt.savefig(str(save_path)+"{}.png".format(spk_ID[i]))
    # # np.save(str(save_path)+"{}.npy".format(spk_ID[i]),log_S)
    imageio.imwrite(str(save_path)+"{}.png".format(spk_ID[i]),S_scaled[::-1])
    



# Load the saved model to make new predictions


model_ft = torch.load('MODEL_LOCATION/final_resnet18_model_with_others_class.pt')
model_ft.eval()


trans = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    # transforms.Resize(32),
    # transforms.CenterCrop(32),
    
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
    ])



# Load images on which predictions to be made


image_datasets_test = datasets.ImageFolder('PATH TO THE LEVEL UP OF SPECTROGRAM FOLDER(e.g. if images are at ./abc/def/ then just mention ./abc/',trans)
                  
dataloaders_test = torch.utils.data.DataLoader(image_datasets_test, batch_size=1, shuffle=False)

f = open("PATH FOR FINAL OUTPUT .TXT FILE/final_prediction_4classes_1.txt", "w+")
with torch.no_grad():
  for i, (image, labels) in enumerate(dataloaders_test):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    image = image.to(device)
    #labels = labels.to(device)
    input = image.view(1, 3, 224, 224)
    outputs = model_ft(input)
    preds = int(torch.argmax(outputs, dim=1).cpu().numpy())
    if (preds ==0):
      preds = 'others'
    if (preds ==1):
      preds = 'PantHoot'
    if (preds ==2):
      preds = 'Drumming'
    if (preds ==3):
      preds = 'scream'
      
    # Uncomment the below section in order to compare the prediction with the ground truth
    #if (labels ==0):
     # labels = 'others'
    #if (labels ==1):
     # labels = 'ph'
    #if (labels ==2):
     # labels ='phtb'
    #if (labels ==3):
     # labels ='sm'
    sample_fname, _ = dataloaders_test.dataset.samples[i]
    #print(sample_fname+ "Prediction :" + preds + " " + "Actual :" + str(labels))
    #print(sample_fname +"\tPred: "+preds)
    f.write("{}, \t{}\n".format(sample_fname, preds))
      
f.close()
      

